In [1]:
import pandas as pd
DelDF = pd.read_csv('Deliveries.csv')
OldDelDF = pd.read_csv('DeliveriesOld.csv')
DaysDF = pd.read_csv('Days.csv')
WeeksDF = pd.read_csv('Weeks.csv')
gpsDF = pd.read_csv('GPS.csv')

In [2]:
def dollars_to_floats(df, columns):
    for x in columns:
        print(x)
        df[x] = df[x].str.replace('$', '')
        df[x] = df[x].str.replace(',', '')
        df[x] = df[x].astype(float)

In [3]:
frames = [OldDelDF, DelDF]
result = pd.concat(frames)

In [4]:
result["Pay"] = result["Pay"].str.replace('$', '')
result["Tip"] = result["Tip"].str.replace('$', '')
result["Total"] = result["Total"].str.replace('$', '')
result["Peak Bonus"] = result["Peak Bonus"].str.replace('$', '')

result["Total"] = result["Total"].astype('float')
result["Pay"] = result["Pay"].astype('float')
result["Tip"] = result["Tip"].astype('float')
result["Peak Bonus"] = result["Peak Bonus"].astype('float')

/tmp/ipykernel_40291/1330755715.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  result["Pay"] = result["Pay"].str.replace('$', '')
/tmp/ipykernel_40291/1330755715.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  result["Tip"] = result["Tip"].str.replace('$', '')
/tmp/ipykernel_40291/1330755715.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  result["Total"] = result["Total"].str.replace('$', '')
/tmp/ipykernel_40291/1330755715.py:4: FutureWarning: The default value of regex will change from True to False in a f

In [5]:
result1 = result[result["Version"] == 1.0]
result2 = result[result["Version"] == 2.0]
result3 = result[result["Version"] == 3.0]


In [6]:
resultAVG = result.groupby(["Restaurant Name"]).agg({"Pay": "mean", "Tip": "mean"})
resultSUM = result.groupby(["Restaurant Name"]).agg({"Pay": "sum", "Tip": "sum"})

resultAVG = resultAVG.round({'Pay': 2, 'Tip': 2})
resultSUM = resultSUM.round({'Pay': 2, 'Tip': 2})

In [7]:
numUntipped = (result["Tip"] == 0.0).sum()
print(numUntipped)
numTipped = (result["Tip"] != 0.0).sum()
print(numTipped)
percTipped = (numTipped/(numUntipped+numTipped))
print(percTipped)

77
597
0.8857566765578635


In [8]:
# 88.58% of customers tip

In [9]:
#Doodle that converts 'Date' attribute to Timestamp object
#Might be easiest way to extract day of the week from the string given in 'Date'

from datetime import date, time

datevar = result.iloc[600]["Date"]
timevar = result.iloc[600]["Start Time"]
print(datevar, " ", timevar)
datevar1 = '2023/' + datevar + " " + timevar
datevar1 = pd.Timestamp(datevar1)
print(datevar1)
print(datevar1.day_name())
print(type(datevar1))

5/27   7:19:00 PM
2023-05-27 19:19:00
Saturday
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [10]:
result.dtypes
#So after parsing string '$##.##' to float we have
#Date which could be a date instead
#STart/End time which could be a time instead
#Miles which should be a float
#Duration which should be an int
#And Rest name and stacked which sort of need to be strings

ID                   int64
Date                object
Restaurant Name     object
Total              float64
Pay                float64
Tip                float64
Peak Bonus         float64
Stacked             object
Version            float64
Start Time          object
Miles               object
End Time            object
Est. Duration       object
dtype: object

In [11]:
#We can convert all dates, but start/end time is a v3 thing, so only convert when version == 3
datetest = result.iloc[600]["Date"] + "/2023"
timetest = result.iloc[600]["Start Time"]
datetest = pd.to_datetime(datetest, format="%m/%d/%Y")
print(type(datetest))
print(datetest.strftime('%m-%d'), timetest)

#So maybe what we want to do, is convert date, start, end to datetime, exclude v1/v2 for start/end related analysis
#now try convert start time to date time
timetest1 = pd.to_datetime(timetest)
print(timetest1)

#interesting, it automatically uses the current date without an existing date
#maybe we just convert both to datetime, then use datetime.date for one and datetime.time for the other

print(timetest1.hour, timetest1.minute, datetest.month, datetest.day)
datetimetest = pd.Timestamp.combine(date(datetest.year, datetest.month, datetest.day), time(timetest1.hour, timetest1.minute))
print(datetimetest)
print(type(datetimetest))

#Okay so that's how we can get the timestamp for date and for time, that leaves miles and duration to check

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
05-27 7:19:00 PM
2023-06-19 19:19:00
19 19 5 27
2023-05-27 19:19:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [12]:
#check miles and duration
a1 = result["Miles"].unique()
print(a1)

a2 = result["Est. Duration"].unique()
print(a2)


[nan '6.2' '2.9' '5.5' '3.1' '3.3' '7.1' '4.84' '5.2' '3.7' '4.7' '3.0'
 '5.9' '5.1' '-' '8.2' '6.1' '7.2' '3.2' '2.6' '2.4' '9.5' '8.3' '2.3'
 '6.9' '4.5' '7.7' '2.5' '5.0' '1.8' '4.0' '7.3' '6.3' '5.3' '7.5' '2.0'
 '3.4' '3.6' '4.2' '6.0' '2.7' '7.4' '10.1' '8.0' '15.9' '7.6' '6.5' '4.4'
 '1.9' '4.3' '4.1' '1.4' '4.8' '1.2' '6.8' '10.2' '12.1']
[nan '30' '20' '28' '34' '25' '35' '15' '26' '18' '29' '32' '-' '41' '33'
 '21' '23' '17' '22' '37' '39' '19' '12' '52' '16' '27' '13' '14' '10']


In [13]:

gpsDF1 = gpsDF[["ID", "Date", "Restaurant Name", "RAW DATA"]].copy()
gpsDF2 = gpsDF1[["Restaurant Name", "RAW DATA"]].copy()
gpsDF2 = gpsDF2.drop_duplicates(subset=["Restaurant Name", "RAW DATA"])
gpsDF2

,Restaurant Name,RAW DATA
0,Erika's Restaurant,Erika's Mexican Food & Seafood Vista
1,Chick-fil-A,Chick-fil-A (01507)
2,Wendy's,Wendy's (8407)
3,Annam Pho and Sandwiches,Annam Pho and Sandwiches (Plaza Dr)
4,Disfruta,Disfruta (Oceanside)
...,...,...
688,Big Bob's Best Pizza,Big Bob's Best Pizza
696,???(tacobell)???,CCC0005121 (3845 Plaza Drive)
703,Del Taco,Del Taco (1305)
704,??(other taco bell),CCC0005121 (695 College Boulevard)


In [14]:
gpsDF2 = gpsDF2.sort_values(by="Restaurant Name", ascending=True)
gpsDF2.to_csv('gpsDF2.csv')

In [15]:
tempDF = result[result["Peak Bonus"] != 0 ]
tempDF.shape
#Well if we only have 45 peak bonus orders that probably isn't enough to make insights from
#But also, I really dont like working at 1am-3am so, we might just drop those insights for now

(45, 13)

In [16]:
tempDF = result[result["Restaurant Name"].duplicated() == True]
tempDF
#shows only restaurants with more than one entry. sort of interesting, about 140 one visits

,ID,Date,Restaurant Name,Total,Pay,Tip,Peak Bonus,Stacked,Version,Start Time,Miles,End Time,Est. Duration
9,11,3/31,Chick-fil-A,7.50,6.50,1.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN
14,16,3/31,Chick-fil-A,8.50,4.00,4.5,0.0,NaN,1.0,NaN,NaN,NaN,NaN
17,19,3/31,PanPan Wok,4.25,3.25,1.0,0.0,18.0,1.0,NaN,NaN,NaN,NaN
19,21,3/31,McDonald's,12.00,4.00,8.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN
20,22,3/31,McDonald's,5.25,3.25,2.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,670,6/4,Annam Pho and Sandwiches,8.00,4.00,4.0,0.0,NaN,3.0,6:02:00 PM,2.5,6:19:00 PM,17
96,671,6/4,Popeyes,7.25,5.25,2.0,0.0,NaN,3.0,7:35:00 PM,4.0,7:49:00 PM,14
98,673,6/4,Urban Plates,11.25,5.25,6.0,0.0,672,3.0,6:54:00 PM,10.2,7:27:00 PM,33
99,674,6/4,Los Tacos,6.75,3.25,3.5,0.0,NaN,3.0,7:54:00 PM,2.0,8:04:00 PM,10


In [17]:
tempDF = result[result["Restaurant Name"].duplicated() == False]
tempDF

,ID,Date,Restaurant Name,Total,Pay,Tip,Peak Bonus,Stacked,Version,Start Time,Miles,End Time,Est. Duration
0,2,3/27,Erika's Restaurant,6.75,5.25,1.50,0.0,NaN,1.0,NaN,NaN,NaN,NaN
1,3,3/27,Chick-fil-A,8.00,4.50,3.50,0.0,NaN,1.0,NaN,NaN,NaN,NaN
2,4,3/27,Wendy's,7.00,3.00,4.00,0.0,NaN,1.0,NaN,NaN,NaN,NaN
3,5,3/27,Annam Pho and Sandwiches,7.00,7.00,0.00,0.0,NaN,1.0,NaN,NaN,NaN,NaN
4,6,3/31,Disfruta,4.75,3.75,1.00,0.0,NaN,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,594,5/26,Little Caesars,6.50,3.50,3.00,0.0,NaN,3.0,-,-,-,-
34,609,5/27,Maans Mediterranean Grill,12.00,5.50,6.50,0.0,598,3.0,8:35:00 PM,7.2,9:14:00 PM,39
80,655,6/3,Chef Tang,27.00,14.00,13.00,0.0,NaN,3.0,6:24:00 PM,15.9,6:58:00 PM,34
88,663,6/3,Wingstop Drive,9.54,6.25,3.29,0.0,NaN,3.0,7:04:00 PM,4.3,7:25:00 PM,21


In [18]:
#Okay so for some reason it gets to 572 then resets to 0 - 100
#is v3 really 100 exactly?
#No, old goes to 574, new starts at 575 and goes to 716
#I guess it would have kept the index from combining olddel and del, so we should try that and rework
result.reset_index(drop=True, inplace=True)

In [19]:
#pd.set_option('display.max_rows', None)
#display(result["Restaurant Name"])
#pd.reset_option('all')

In [20]:
#Okay thats sensible, the ID starts at 2 and the index starts at 0 and this isn't the latest csv
#pd.set_option('display.max_rows', None)
#display(result["Restaurant Name"].duplicated(keep=False) == False)
#pd.reset_option('all')

In [21]:
#Okay so duplicated has two parameters, the second of which is keep
#We want keep=false, so that all restaurant names that exist more than once are flagged as duplicates
#keep=first is default, which seems silly over keep=false being default
#but keep=first doesn't flag the first as a duplicate
#I can see that being useful in certain contexts
#and strictly speaking, the rest would be duplicates of the original, which makes it fair to not consider the first
#as a duplicate
#but for what we want, keep=False 

In [22]:
tempDF = result[result["Restaurant Name"].duplicated(keep=False) == False]

#So this is all the restaurants we've only gone to, why do I keep saying we
#This is all the places I've picked up from only once
#Except for the typos that haven't been updated out yet

#We're looking for highest and lowest tipping restaurants, so we actually want to look for the opposite of this set

In [23]:
tempDF = result[result["Restaurant Name"].duplicated(keep=False) == True]


#And we want to drop where tip == 0, actually no I think 0 tip should count as a tip value for lowest tipping customers
#So aggregate on restaurant name and sum for tip

In [24]:
tempDF = tempDF.groupby(["Restaurant Name"]).agg({"Pay": "mean", "Tip": "mean"})
tempDF = tempDF.round({'Pay': 2, 'Tip': 2})
tempDF.sort_values("Tip", ascending=False)

,Pay,Tip
Restaurant Name,,
Albertsons,4.58,12.39
Hunter Steakhouse,10.00,11.50
Chin's Szechwan,5.88,9.25
Mountain Mike's Pizza,4.88,8.25
The Privateer Coal Fire Pizza,5.12,8.00
...,...,...
Panda Express,4.21,1.75
Carl's Jr.,4.38,1.50
Sizzler,12.75,1.25


In [25]:
#So we see car parts and flower orders as the only actual 0 tip
#Car parts seem to be a weird outsourced thing that runs through the delivery service
#FTD is a flower shop thing and I've only done two of those orders but uh
#Yeah no tip 
#On the other hand, grocery orders tip well, which makes sense because they are super obnoxious to handle
#compared to one bag of food from a store
#Hunters is probably the most expensive place i've picked up from, chin's also isn't especially cheap for the area
#And we see large avg tip values from two pizza places in the top 5
#I've done a few panda express orders so its a bit disappointing to see that value so low

In [26]:
#Overall hourly rate, Total from TOTALS / Dash
#We haven't converted to datetime yet so this will just be a rough draft
totalPay = WeeksDF.loc[10]["Total"]
print(totalPay)

totalPay = totalPay.replace('$', '')
totalPay = totalPay.replace(',', '')
totalPay = float(totalPay)

print(totalPay)

totalTime = WeeksDF.loc[10]["Dash"]
#This should be changed to take the full string, pass everything through to the first : as hours
#and everything between the first and second : as minutes
totalTimeH = totalTime[0:3]
totalTimeM = totalTime[4:6]
#okay so it includes the first position and does not include the second position, so its a [x, y) situation
totalTimeH = float(totalTimeH)
totalTimeM = float(totalTimeM)
print(totalTimeM)
totalTimeM = totalTimeM/60
print(totalTimeM)
totalTime = totalTimeH + totalTimeM
print(totalTime)

hourlyRate = totalPay/totalTime
print("$", hourlyRate, "per hour.")

$7,112.19
7112.19
59.0
0.9833333333333333
294.98333333333335
$ 24.110480818125314 per hour.


In [27]:
#Best and worst 'pretotal' vs 'active time'
WeeksDF["Pre Total"] = WeeksDF["Pre Total"].str.replace('$', '')
WeeksDF["Pre Total"] = WeeksDF["Pre Total"].str.replace(',', '')
WeeksDF["Pre Total"] = WeeksDF["Pre Total"].astype(float)
WeeksDF


#So float(var) converts, but series.astype(float) is for series in df

/tmp/ipykernel_40291/1848943084.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  WeeksDF["Pre Total"] = WeeksDF["Pre Total"].str.replace('$', '')


,Dates,Base,Tip,Adj pay,Other,Pre Total,Total,Active,Dash,Deliveries,Version
0,3/27 - 4/2,$108.50,$88.00,$40.96,$0.00,196.50,$237.46,6:42:00,10:55:00,22,1.0
1,4/3 - 4/9,$264.25,$230.69,$58.17,$0.00,494.94,$553.11,14:41:00,22:02:00,55,1.0
2,4/10 - 4/16,$130.75,$96.13,$61.07,$0.00,226.88,$287.95,8:38:00,11:10:00,28,1.0
3,4/17 - 4/23,$382.50,$248.41,$105.05,$0.00,630.91,$735.96,21:20:00,32:18:00,68,1.0
4,4/24 - 4/30,$430.00,$299.73,$138.76,$2.62,729.73,$871.11,25:30:00,41:10:00,84,1.0
5,5/1 - 5/7,$618.50,$445.32,$121.74,$0.00,1063.82,"$1,185.56",32:24:00,50:40:00,114,1.0
6,5/8 - 5/14,$758.25,$567.84,$155.48,$6.25,1326.09,"$1,487.82",40:35:00,58:08:00,138,2.0
7,5/15 - 5/21,$303.75,$239.25,$99.27,$0.00,543.00,$642.27,17:32:00,25:24:00,64,2.0
8,5/22 - 5/28,$205.50,$189.19,$63.63,$0.00,394.69,$458.32,11:39:00,16:40:00,38,3.0
9,5/29 - 6/4,$342.75,$309.88,$115.30,$0.00,652.63,$767.93,19:24:00,26:32:00,63,3.0


In [28]:
tempX = WeeksDF.iloc[0]["Active"]
print(tempX)
tempX1 = tempX
tempX2 = tempX
tempX1 = tempX[:tempX.index(":")]
#print(tempX1)
tempX2 = tempX[tempX.index(":")+1:tempX.index(":")+3]
#print(tempX)
#print(tempX1)
#print(tempX2)
tempX1 = float(tempX1)
tempX2 = float(tempX2)
tempX2 = tempX2/60
tempX = tempX1 + tempX2
print(tempX)
tempY = float(WeeksDF.iloc[0]["Active"][:WeeksDF.iloc[0]["Active"].index(":")]) + float(WeeksDF.iloc[0]["Active"][WeeksDF.iloc[0]["Active"].index(":")+1:WeeksDF.iloc[0]["Active"].index(":")+3])/60
print(tempY)

#This was the greediest in line, just iterate through the columns and preserve your sanity please
#WeeksDF["Active"] = float(WeeksDF["Active"][:WeeksDF["Active"].index(":")]) + float(WeeksDF["Active"][WeeksDF["Active"].index(":")+1:WeeksDF["Active"].index(":")+3])/60
WeeksDF

6:42:00
6.7
6.7


,Dates,Base,Tip,Adj pay,Other,Pre Total,Total,Active,Dash,Deliveries,Version
0,3/27 - 4/2,$108.50,$88.00,$40.96,$0.00,196.50,$237.46,6:42:00,10:55:00,22,1.0
1,4/3 - 4/9,$264.25,$230.69,$58.17,$0.00,494.94,$553.11,14:41:00,22:02:00,55,1.0
2,4/10 - 4/16,$130.75,$96.13,$61.07,$0.00,226.88,$287.95,8:38:00,11:10:00,28,1.0
3,4/17 - 4/23,$382.50,$248.41,$105.05,$0.00,630.91,$735.96,21:20:00,32:18:00,68,1.0
4,4/24 - 4/30,$430.00,$299.73,$138.76,$2.62,729.73,$871.11,25:30:00,41:10:00,84,1.0
5,5/1 - 5/7,$618.50,$445.32,$121.74,$0.00,1063.82,"$1,185.56",32:24:00,50:40:00,114,1.0
6,5/8 - 5/14,$758.25,$567.84,$155.48,$6.25,1326.09,"$1,487.82",40:35:00,58:08:00,138,2.0
7,5/15 - 5/21,$303.75,$239.25,$99.27,$0.00,543.00,$642.27,17:32:00,25:24:00,64,2.0
8,5/22 - 5/28,$205.50,$189.19,$63.63,$0.00,394.69,$458.32,11:39:00,16:40:00,38,3.0
9,5/29 - 6/4,$342.75,$309.88,$115.30,$0.00,652.63,$767.93,19:24:00,26:32:00,63,3.0


In [29]:

for index, row in WeeksDF.iterrows():
    #print(row["Active"])
    active = row["Active"]
    activeH = float(active[:active.index(":")])
    activeM = round(float(active[active.index(":")+1:active.index(":")+3])/60, 2)
    active = activeH + activeM
    #print(active)
    WeeksDF.at[index, "Active"] = active

In [30]:
#Okay now we have pre total and active in a format that allows for good math
WeeksDF["pretotalactiveratio"] = WeeksDF["Pre Total"]/WeeksDF["Active"]

In [31]:
WeeksDF.sort_values(by=["pretotalactiveratio"])
#right well its kind of interesting to see totals in the middle
#and it also looks like May had better ratios, which makes it seem like I'm getting better?
#better isn't a good characterization
#pre total is the money before adjustment, its just base pay + tip 
#so may having better base pay + tip / time actually spent on deliveries is interesting

,Dates,Base,Tip,Adj pay,Other,Pre Total,Total,Active,Dash,Deliveries,Version,pretotalactiveratio
2,4/10 - 4/16,$130.75,$96.13,$61.07,$0.00,226.88,$287.95,8.63,11:10:00,28,1.0,26.289687
4,4/24 - 4/30,$430.00,$299.73,$138.76,$2.62,729.73,$871.11,25.5,41:10:00,84,1.0,28.616863
0,3/27 - 4/2,$108.50,$88.00,$40.96,$0.00,196.50,$237.46,6.7,10:55:00,22,1.0,29.328358
3,4/17 - 4/23,$382.50,$248.41,$105.05,$0.00,630.91,$735.96,21.33,32:18:00,68,1.0,29.578528
7,5/15 - 5/21,$303.75,$239.25,$99.27,$0.00,543.00,$642.27,17.53,25:24:00,64,2.0,30.975471
10,TOTALS:,"$3,544.75","$2,714.44",$844.13,$8.87,6259.19,"$7,112.19",198.42,294:59:00,674,NaN,31.545157
6,5/8 - 5/14,$758.25,$567.84,$155.48,$6.25,1326.09,"$1,487.82",40.58,58:08:00,138,2.0,32.678413
5,5/1 - 5/7,$618.50,$445.32,$121.74,$0.00,1063.82,"$1,185.56",32.4,50:40:00,114,1.0,32.833951
9,5/29 - 6/4,$342.75,$309.88,$115.30,$0.00,652.63,$767.93,19.4,26:32:00,63,3.0,33.640722
1,4/3 - 4/9,$264.25,$230.69,$58.17,$0.00,494.94,$553.11,14.68,22:02:00,55,1.0,33.715259


In [32]:
#We want to Miles vs Total, miles only exists in v3
#display(DelDF)
v3DF = DelDF.copy()

v3DF["Total"] = v3DF["Total"].str.replace('$', '')
v3DF["Total"] = v3DF["Total"].str.replace(',', '')
v3DF["Total"] = v3DF["Total"].astype(float)
v3DF = v3DF[v3DF.Miles!="-"]
v3DF["Miles"] = v3DF["Miles"].astype(float)

#display(v3DF)
v3DF = v3DF.groupby(["Restaurant Name"]).agg({"Total": "sum", "Miles": "sum"})
v3DF = v3DF.round({'Pay': 2, 'Tip': 2})

v3DF["DollarMilesRatio"] = v3DF["Total"]/v3DF["Miles"]
v3DF.sort_values(by="DollarMilesRatio")

#display(v3DF)

#no no, we want to look at aggregate values, so sum pay vs sum miles

/tmp/ipykernel_40291/1666305466.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  v3DF["Total"] = v3DF["Total"].str.replace('$', '')


,Total,Miles,DollarMilesRatio
Restaurant Name,,,
Urban Plates,21.50,20.40,1.053922
7-Eleven,20.00,18.40,1.086957
Colimas Mexican Food,6.25,5.30,1.179245
Gianni's Pizza,11.50,9.50,1.210526
Dashmart,9.75,7.60,1.282895
Jack in the Box,20.25,15.50,1.306452
Pizza on tap,13.50,10.10,1.336634
China Fusion,39.00,26.40,1.477273
Felix's BBQ With Soul,20.50,13.80,1.485507


In [33]:
#well max() seems useful
DaysDF.max()
#DaysDF["Deliveries"]

Date                    6/4
Total                $86.13
Base                 $99.00
Tip                  $94.03
Start (24)            18:58
End (24)               2:32
Active time (h:m)      9:40
Dash time (h:m)        9:41
Offers                   32
Deliveries               33
Version                   3
Pauses                    2
dtype: object

In [34]:
#So we need to convert $ to float 
#Really we just need to make a function to convert time values to hour in float format and string values to float

In [35]:
TipAvg = result["Tip"].mean()
print(TipAvg)
TipAvg2 = result[result["Tip"] !=0]["Tip"].mean() #weird
print(TipAvg2)

4.027359050445104
4.54680067001675


In [36]:
tempDF = DelDF.copy()
listToPass = ["Total", "Pay", "Tip", "Peak Bonus"]

dollars_to_floats(tempDF, listToPass)
tempDF.dtypes
tempDF['Miles'].unique()

Total
Pay
Tip
Peak Bonus


/tmp/ipykernel_40291/57653493.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')


array(['6.2', '2.9', '5.5', '3.1', '3.3', '7.1', '4.84', '5.2', '3.7',
       '4.7', '3.0', '5.9', '5.1', '-', '8.2', '6.1', '7.2', '3.2', '2.6',
       '2.4', '9.5', '8.3', '2.3', '6.9', '4.5', '7.7', '2.5', '5.0',
       '1.8', '4.0', '7.3', '6.3', '5.3', '7.5', '2.0', '3.4', '3.6',
       '4.2', '6.0', '2.7', '7.4', '10.1', '8.0', '15.9', '7.6', '6.5',
       '4.4', '1.9', '4.3', '4.1', '1.4', '4.8', '1.2', '6.8', '10.2',
       '12.1'], dtype=object)

In [37]:
print(tempDF.shape)
tempDF = tempDF[tempDF['Miles'] != '-']
print(tempDF.shape)
tempDF['Miles'].unique()



#okay there's just the one dash so we drop when it equals dash

(101, 13)
(100, 13)


array(['6.2', '2.9', '5.5', '3.1', '3.3', '7.1', '4.84', '5.2', '3.7',
       '4.7', '3.0', '5.9', '5.1', '8.2', '6.1', '7.2', '3.2', '2.6',
       '2.4', '9.5', '8.3', '2.3', '6.9', '4.5', '7.7', '2.5', '5.0',
       '1.8', '4.0', '7.3', '6.3', '5.3', '7.5', '2.0', '3.4', '3.6',
       '4.2', '6.0', '2.7', '7.4', '10.1', '8.0', '15.9', '7.6', '6.5',
       '4.4', '1.9', '4.3', '4.1', '1.4', '4.8', '1.2', '6.8', '10.2',
       '12.1'], dtype=object)

In [38]:
import numpy as np

#testingDF["Miles"] = tempDF["Miles"].applymap(np.isreal)
#tempDF["Miles"].applymap(lambda x: isinstance(x, (int, float)))
tempDF2 = tempDF[pd.to_numeric(tempDF['Miles'], errors='coerce').isnull()]['Miles'].unique()

In [39]:
tempDF['Miles'] = tempDF['Miles'].astype(float)
tempDF['Miles']

0       6.2
1       2.9
2       5.5
3       3.1
4       3.3
       ... 
96      4.0
97     10.2
98     10.2
99      2.0
100    12.1
Name: Miles, Length: 100, dtype: float64

In [40]:
#DATETIME DOODLE

date = tempDF.iloc[-1]["Date"]
starttime = tempDF.iloc[-1]["Start Time"]
endtime = tempDF.iloc[-1]["End Time"]
print(date, starttime, endtime)
date = '2023/' + datevar + " " + starttime
date = pd.Timestamp(date)
starttime = date
endtime = '2023/' + datevar + " " + endtime
endtime = pd.Timestamp(endtime)
print("\n", date)
print(date.day_name())
print(starttime.minute)
print(endtime.minute)
#print(type(date))



#We can convert all dates, but start/end time is a v3 thing, so only convert when version == 3
#datetest = result.iloc[600]["Date"] + "/2023"
#timetest = result.iloc[600]["Start Time"]
#datetest = pd.Timestamp(datetest)
#datetest = pd.to_datetime(datetest, format="%m/%d/%Y")
#print(type(datetest))
#print(datetest.strftime('%m-%d'), timetest)
#type(datetest)

#So maybe what we want to do, is convert date, start, end to datetime, exclude v1/v2 for start/end related analysis
#now try convert start time to date time
#timetest1 = pd.to_datetime(timetest)
#print(timetest1)

#interesting, it automatically uses the current date without an existing date
#maybe we just convert both to datetime, then use datetime.date for one and datetime.time for the other

#datetimetest = pd.Timestamp.combine(date(datetest), time(timetest1))
#print(datetimetest)
#print(timetest1.hour, timetest1.minute, datetest.month, datetest.day)
#datetimetest = pd.Timestamp.combine(date(datetest.year, datetest.month, datetest.day), time(timetest1.hour, timetest1.minute))
#print(datetimetest)
#print(type(datetimetest))

#Okay so that's how we can get the timestamp for date and for time, that leaves miles and duration to check

6/4 6:21:00 PM 6:51:00 PM

 2023-05-27 18:21:00
Saturday
21
51


In [41]:
#DaysDF["Date"] = pd.Timestamp('2023/' + DaysDF["Date"])
#display(DaysDF)
#for index, row

for index, row in DaysDF.iterrows():
    #print(row["Active"])
    date = pd.Timestamp('2023/' + row["Date"])
    DaysDF.at[index, "Date"] = date
    


In [42]:
DaysDF["Date"] = pd.to_datetime(DaysDF["Date"])
DaysDF["Day of Week"] = DaysDF["Date"].dt.day_name()
#display(DaysDF)
#DaysDF.dtypes

In [43]:
for index, row in DaysDF.iterrows():
    #print(row["Active"])
    active = row["Dash time (h:m)"]
    activeH = float(active[:active.index(":")])
    activeM = round(float(active[active.index(":")+1:active.index(":")+3])/60, 2)
    active = activeH + activeM
    #print(active)
    DaysDF.at[index, "Dash time (h:m)"] = active


In [44]:
DaysDF.dtypes

Date                 datetime64[ns]
Total                        object
Base                         object
Tip                          object
Start (24)                   object
End (24)                     object
Active time (h:m)            object
Dash time (h:m)              object
Offers                        int64
Deliveries                    int64
Version                       int64
Pauses                       object
Day of Week                  object
dtype: object

In [45]:
DaysDF["Dash time (h:m)"] = DaysDF["Dash time (h:m)"].astype(float)
DaysDF.dtypes

dollars_to_floats(DaysDF, ["Total", "Base", "Tip"])

DaysDF.dtypes

Total
Base
Tip


/tmp/ipykernel_40291/57653493.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[x] = df[x].str.replace('$', '')


Date                 datetime64[ns]
Total                       float64
Base                        float64
Tip                         float64
Start (24)                   object
End (24)                     object
Active time (h:m)            object
Dash time (h:m)             float64
Offers                        int64
Deliveries                    int64
Version                       int64
Pauses                       object
Day of Week                  object
dtype: object

In [46]:
DaysDF = DaysDF.groupby(["Day of Week"]).agg(
    {"Dash time (h:m)": "sum", "Total": "sum", "Tip": "sum", "Deliveries ": "sum"})
DaysDF = DaysDF.round({'Dash time (h:m)': 2, 'Total': 2, "Tip": 2})
display(DaysDF)

#mean Tip is mean total tip not mean tip on the day of, which requires more work but its dinner time so I'll ignore it for now

#v3DF["DollarMilesRatio"] = v3DF["Total"]/v3DF["Miles"]
#v3DF.sort_values(by="DollarMilesRatio")




,Dash time (h:m),Total,Tip,Deliveries
Day of Week,,,,
Friday,86.48,1880.74,839.24,201
Monday,7.02,135.97,56.97,14
Saturday,68.52,1563.17,699.17,157
Sunday,28.06,665.73,306.98,69
Thursday,56.86,1167.71,484.21,134
Wednesday,48.00,845.87,327.87,99


In [47]:
DaysDF["$/hr"] = DaysDF["Total"]/DaysDF["Dash time (h:m)"]
display(DaysDF)

,Dash time (h:m),Total,Tip,Deliveries,$/hr
Day of Week,,,,,
Friday,86.48,1880.74,839.24,201,21.747687
Monday,7.02,135.97,56.97,14,19.368946
Saturday,68.52,1563.17,699.17,157,22.813339
Sunday,28.06,665.73,306.98,69,23.725232
Thursday,56.86,1167.71,484.21,134,20.536581
Wednesday,48.00,845.87,327.87,99,17.622292


In [48]:
DaysDF["AvgTip"] = DaysDF["Tip"]/DaysDF["Deliveries "]
display(DaysDF)

,Dash time (h:m),Total,Tip,Deliveries,$/hr,AvgTip
Day of Week,,,,,,
Friday,86.48,1880.74,839.24,201,21.747687,4.175323
Monday,7.02,135.97,56.97,14,19.368946,4.069286
Saturday,68.52,1563.17,699.17,157,22.813339,4.453312
Sunday,28.06,665.73,306.98,69,23.725232,4.448986
Thursday,56.86,1167.71,484.21,134,20.536581,3.613507
Wednesday,48.00,845.87,327.87,99,17.622292,3.311818
